In [ ]:
# Importing Libraries
import tensorflow as tf
from keras.models import Sequential
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.datasets import mnist
from keras.callbacks import ModelCheckpoint
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Activation, Dropout

In [ ]:
# VGG16 Model
def build_vgg16(input_shape, num_classes):
    model = Sequential([
        # Block 1
        Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2), strides=(2, 2)),

        # Block 2
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2), strides=(2, 2)),

        # Block 3
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2), strides=(2, 2)),

        # Block 4
        Conv2D(512, (3, 3), activation='relu', padding='same'),
        Conv2D(512, (3, 3), activation='relu', padding='same'),
        Conv2D(512, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2), strides=(2, 2)),

        Flatten(),
        Dense(4096, activation='relu'),
        Dense(4096, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])

    return model

In [ ]:
# Load the MNIST dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Preprocess the data
input_shape = (28, 28, 1)
num_classes = 10
train_images = train_images.reshape(-1, 28, 28, 1).astype('float32') / 255.0
test_images = test_images.reshape(-1, 28, 28, 1).astype('float32') / 255.0
train_labels = to_categorical(train_labels, num_classes)
test_labels = to_categorical(test_labels, num_classes)

# Split the training data into training and validation sets
# You can use any method to split the data, such as train_test_split from sklearn
from sklearn.model_selection import train_test_split
train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels, test_size=0.1, random_state=42)

# Build the VGG16-like model for MNIST
model = build_vgg16(input_shape, num_classes)
model.summary();

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Define the ModelCheckpoint callback to save the best model
checkpoint_filepath = 'best_model.h5'
model_checkpoint_callback = ModelCheckpoint(filepath=checkpoint_filepath,
                                            save_best_only=True,
                                            monitor='val_accuracy',
                                            mode='max',
                                            verbose=1)

# Fit the model to the training data with validation data and ModelCheckpoint callback
history = model.fit(train_images, train_labels, epochs=10, batch_size=128, validation_data=(val_images, val_labels),
                    callbacks=[model_checkpoint_callback])

# Evaluate the best model on the test data
best_model = tf.keras.models.load_model(checkpoint_filepath)
test_loss, test_accuracy = best_model.evaluate(test_images, test_labels, verbose=2)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_40 (Conv2D)          (None, 28, 28, 64)        640       
                                                                 
 conv2d_41 (Conv2D)          (None, 28, 28, 64)        36928     
                                                                 
 max_pooling2d_16 (MaxPooli  (None, 14, 14, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_42 (Conv2D)          (None, 14, 14, 128)       73856     
                                                                 
 conv2d_43 (Conv2D)          (None, 14, 14, 128)       147584    
                                                                 
 max_pooling2d_17 (MaxPooli  (None, 7, 7, 128)         0         
 ng2D)                                                

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


422/422 [==============================] - 24s 49ms/step - loss: 1.0726 - accuracy: 0.5983 - val_loss: 0.1072 - val_accuracy: 0.9692
Epoch 2/10
422/422 [==============================] - ETA: 0s - loss: 0.0886 - accuracy: 0.9756
Epoch 2: val_accuracy improved from 0.96917 to 0.98450, saving model to best_model.h5
422/422 [==============================] - 21s 49ms/step - loss: 0.0886 - accuracy: 0.9756 - val_loss: 0.0556 - val_accuracy: 0.9845
Epoch 3/10
421/422 [============================>.] - ETA: 0s - loss: 0.0605 - accuracy: 0.9833
Epoch 3: val_accuracy improved from 0.98450 to 0.98600, saving model to best_model.h5
422/422 [==============================] - 20s 48ms/step - loss: 0.0605 - accuracy: 0.9833 - val_loss: 0.0512 - val_accuracy: 0.9860
Epoch 4/10
421/422 [============================>.] - ETA: 0s - loss: 0.0472 - accuracy: 0.9870
Epoch 4: val_accuracy did not improve from 0.98600
422/422 [==============================] - 19s 46ms/step - loss: 0.0473 - accuracy: 0.9870